### Custom Homography Tutorial

Here, we will go over how to create custom Homography transformations to images on an in-depth level. 

#### Sources

1. Tutorial on making plots interactive: https://www.geeksforgeeks.org/make-3d-interactive-matplotlib-plot-in-jupyter-notebook/
2. Explanation of how homographies work: https://docs.opencv.org/4.x/d9/dab/tutorial_homography.html
3. 2x2 Image Transformations Explanation Video: https://youtu.be/K2XLXlyPqCA
4. 3x3 Image Transformations Explanation Video: https://www.youtube.com/watch?v=B8kMB6Hv2eI
5. Computing Homographies for Image Stitching Video: https://www.youtube.com/watch?v=l_qjO4cM74o
6. How to create 3D Surfaces Tutorial: https://www.geeksforgeeks.org/three-dimensional-plotting-in-python-using-matplotlib/
7. Image Warping without `Skimage`: https://stackoverflow.com/questions/70311373/applying-homography-transformation-in-python-without-using-opencv
8. Applying homographies with OpenCV: https://learnopencv.com/homography-examples-using-opencv-python-c/
9. Homographies with Image Processing: https://mattmaulion.medium.com/homography-transform-image-processing-eddbcb8e4ff7#:~:text=Homography%2C%20also%20referred%20to%20as,in%20a%20homogenous%20coordinates%20space

### Import the required Python libraries for this notebook.

In [4]:
# Required commands to show interactive 3D plots
%matplotlib notebook
%matplotlib widget
%matplotlib inline

In [5]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import functools
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
from sklearn.preprocessing import normalize

import math
import numpy.matlib as npm
import cv2

### Let's start by creating some functions that will allow us to apply image transformations on custom images!

The functions below allow us to transform objects of interest from a 3D space to a 2D space. This is only half the process since we want to transform objects from a 2D plane to another 2D plane, just from a different perspective (i.e. rotation, warping, etc.). 

We consider a range of different transformations that we can apply to an image to mimic the effects of utilizing the imported `Skimage` transformation functions.

### First, let's define functions that will help us plot 2D objects utilizing homographies.

In [6]:
def get_rid_of_z_dimension(array_3D):
    '''
        This function removes the last dimension of a 3D input space to reflect the 3D -> 2D image transformation notion. 
        Recall that the overall tranformation requires an object to go through 2D -> 3D -> 2D spaces, so this function actually 
        performs the second half of this transition.

        Params: array_3D (a 3D tensor array of the form nx3).
        Returns: a 2D array of the same array (minus the last dimnesion) in the form nx2.
    '''

    return np.delete(array_3D, -1, axis=-1)


def plot_tensor(tensor):
    '''
        This function plots a tensor that can be in 2D or 3D form.

        Params: tensor (a 2D or 3D tensor - can be whichever).
        Returns: None (but plots the input tensor).
    '''

    # Plot color points onto a 3D space
    cmap_colors = [(0, 'red'), (1, 'blue'), (1, 'green')]
    custom_cmap = LinearSegmentedColormap.from_list('custom', cmap_colors)

    num_dimensions = 2

    num_points = int(math.sqrt(tensor.shape[0]))

    # Custom threshold
    lowest_color_value = 0.2
    highest_color_value = 0.8

    x_color = np.linspace(lowest_color_value, highest_color_value/num_dimensions, num_points)
    y_color = np.linspace(lowest_color_value, highest_color_value/num_dimensions, num_points)

    # Meshgrid allows us to compute gradients
    X_color, Y_color = np.meshgrid(x_color, y_color)

    # Combine data for both dimensions into a single variable
    combined_data = X_color + Y_color  # You can use a different combination as needed

    # Apply the custom colormap to the combined data
    color = custom_cmap(combined_data)

    # Make sure the color is in the right form (num of data points x 4), so we have a 
    # color for each datapoint
    color_new = color.reshape((-1, 4))


    if len(tensor.shape) == 2:
        # Create a figure and axis
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')


        ######################################################
        ##############$$     3D Tensor     ###################
        ######################################################

        if tensor.shape[-1] == 3:

            # Display the computed points
            ax.scatter(tensor[:,0], tensor[:,1], tensor[:,2], c=color_new, cmap=custom_cmap, s=50, marker='o')

            # # Set the 3D axis limits so we can view everything
            # min_val = min(tensor.flatten())
            # max_val = max(tensor.flatten())

            # ax.set_xlim(min_val, max_val)
            # ax.set_ylim(min_val, max_val)
            # ax.set_zlim(min_val, max_val)

            plt.show()


        ######################################################
        ##############$$     2D Tensor     ###################
        ######################################################

        elif tensor.shape[-1] == 2:
            # Create a figure and axis
            fig = plt.figure()
            ax = fig.add_subplot(111)
            
            # Display the computed points
            ax.scatter(tensor[:,0], tensor[:,1], c=color_new, cmap=custom_cmap, s=50, marker='o')

            # # Set the 3D axis limits so we can view everything
            # min_val = min(tensor.flatten())
            # max_val = max(tensor.flatten())

            # ax.set_xlim(min_val, max_val)
            # ax.set_ylim(min_val, max_val)

            plt.show()
    return

### Next, let's define functions that illustrate the specific transformations we can apply to our input image(s)!

#### Rotation

In [7]:
def create_rotation_matrix_clockwise(degrees):
    '''
        This function creates a rotation matrix to which we can apply to images in order to rotate
        them counter-clockwise by some degree.

        Params: degrees (the number of degrees to specify the extent of our rotation).
        Returns: a matrix that represents the rotation of an image.
    '''

    radians = math.radians(degrees)     # Convert degrees to radians

    cos_theta = math.cos(radians)
    sin_theta = math.sin(radians)

    # Compute the rotation matrix
    rotation_matrix = np.array([[cos_theta, -sin_theta, 0],
                                [sin_theta, cos_theta, 0],
                                [0, 0, 1]])

    return rotation_matrix

#### Translation

In [8]:
def create_translation_matrix(dx, dy, dz = 0, dims = 2):
    '''
        This function creates a translation matrix to which we can apply to images by specifying the extent of 
        the image's x, y, and z displacement.

        Params: dx (the change in displacement in the x-direction).
                dy (the change in displacement in the y-direction).
                dz (the change in displacement in the z-direction).
                dims (the number of dimensions for our image of interest).
        Returns: a matrix that represents the translation of an image.
    '''
    
    # 2D Image Translation
    if dims == 2:
        translation_matrix = np.array([[1, 0, dx], 
                                       [0, 1, dy], 
                                       [0, 0, 1]])
    
    # 3D Image Translation
    elif dims == 3:
        translation_matrix = np.array([[1, 0, 0, dx],
                                       [0, 1, 0, dy],
                                       [0, 0, 1, dz], 
                                       [0, 0, 0, 1]])

    return translation_matrix

#### Scale

In [10]:
def create_scale_matrix(scaleX, scaleY, dims = 2):
    '''
        This function creates a scale matrix to which we can can apply to images by specifying the 
        amount of scale to the image's x and y axes.

        Params: scaleX (the amount of scale in the image's x axis).
                scaleY (the amount of scale in the image's y axis).
                dims (the number of dimensions for our image of interest).
        Returns: a matrix that represents the scale of an image.
    '''

    # Directly compute the scale matrix
    scale_matrix = np.array([[scaleX, 0, 0],
                             [0, scaleY, 0],
                             [0, 0, 1]])

    return scale_matrix

#### Skew

In [9]:
def create_skew_matrix(skew_amt, dims = 2):
    '''
        This function creates a skew matrix to which we can apply to images by specifying
        the extent of the skew in the horizontal and vertical directions.

        Params: skew_amt (the amount of skew specified as a numerical value).
                dims (the number of dimensions for our image of interest).
        Returns: a matrix that represents the skew of an image (one for horizontal, one for vertical).
    '''

    # Horizontal Skew
    skew_matrix_X = np.array([[1, skew_amt, 0],
                            [0, 1, 0],
                            [0, 0, 1]])
    
    # Vertical Skew
    skew_matrix_Y = np.array([[1, 0, 0],
                              [skew_amt, 1, 0],
                              [0, 0, 1]])
    
    return skew_matrix_X, skew_matrix_Y

#### Warp

In [11]:
def create_warp_matrix(warp_x, warp_y, dims=2):
    '''
        This function creates a warp matrix to which we can apply to images by specifying
        the amount of warping in the x and y axes.

        Params: warp_x (the amount of warping in the horizontal direction).
                warp_y (the amount of warping in the vertical direction).
                dims (the number of dimensions for our image of interest).
        Returns: a matrix that represents the warping of an image.
    '''

    # Compute the warp matrix
    warp_matrix = np.array([[1, 0 , 0], 
                            [0, 1, 0],
                            [warp_x, warp_y, 1]])
    
    return warp_matrix

### We have our functions that apply various different transformations to images. Now, we can create functions that apply a transformation to an image! 

This makes it easier to read and understand what is going on rather than simplying creating long, chunked-up pieces of code.

In [18]:
def apply_transformation(tensor, transformation):
    '''
        Applies a transformation to a particular tensor (image tensor) and returns the transformed tensor.

        Params: tensor (nx2 or nx3 tensor that represents our image).
                transformation (a specific transformation matrix specified by the transformation functions defined above).
        Returns: another version of the input tensor, this time with the applied transformation; should retain initial dimensions.
    '''

    # Print for debugging purposes
    print("transformation.shape: ", transformation.shape)
    print("input.shape: ", tensor.shape)

    return np.matmul(transformation, tensor.T).T

In [13]:
def create_affine_transformation():
    '''
        This function creates an affine transformation matrix that can be applied to images.
        It takes in individual transformation matrices, combines their effects, and applies the resulting
        matrix to images of interest.

        Params: None (instead, the functions that generate individual transformation matrices are called).
        Returns: an affine transformation matrix with the combined effects of the individual transformations.
    '''
    
    # Rotation
    rotation_matrix = create_rotation_matrix_clockwise(45)      # Degrees are chosen by the user

    # Translation
    translation_matrix = create_translation_matrix(5, 10)        # dx, dy, and dz are chosen by the user

    # Scale
    scaling_matrix = create_scale_matrix(1, 2)                  # scaleX, scaleY are chosen by the user

    # Skew
    skew_matrix_X, skew_matrix_Y = create_skew_matrix(0)        # Skew amount is chosen by the user


    # Debugging print statements below

    # print(np.matmul(rotation_matrix, translation_matrix))

    # print("Rotation matrix")
    # print(rotation_matrix)
    # print("Translation matrix")
    # print(translation_matrix)
    # print("scaling matrix")
    # print(scaling_matrix)
    # print("skew matrices")
    # print(skew_matrix_X)
    # print(skew_matrix_Y)

    # print(rotation_matrix.shape)
    # print(translation_matrix.shape)
    # print(scaling_matrix.shape)
    # print(skew_matrix_X.shape)
    # print(skew_matrix_Y.shape)


    # Combine the individual effects of transformations to create affine effect
    affine_transformation = functools.reduce(np.matmul, [translation_matrix, skew_matrix_X, scaling_matrix, rotation_matrix])

    return affine_transformation

In [20]:
def create_homography_transformation():
    '''
        This function creates a custom homography matrix (3x3) by combining the effects of the rotation, translation, 
        scaling, skew, and warping transformations.

        Note: the input values for each of these transformations must be inputted manually by the user to receive their
              desired image transformation (aka affine transformation matrix).
    '''
    
    # Rotation
    rotation_matrix = create_rotation_matrix_clockwise(90)

    # Translation
    translation_matrix = create_translation_matrix(0, 0)

    # Scale
    scaling_matrix = create_scale_matrix(1, 1)

    # Skew
    skew_matrix_X, skew_matrix_Y = create_skew_matrix(0)

    # Warp
    warp_matrix = create_warp_matrix(1, 3)

    # Combine the individual effects of transformations to create homography effect
    homography = functools.reduce(np.matmul, [translation_matrix, skew_matrix_X, scaling_matrix, rotation_matrix, warp_matrix])

    return homography

#### Let's try using the functions we just created in a simple application!

In [23]:
# The number of points along the x and y directions in our plane
num_points = 50

# Set the boundaries of the square we are creating (these constants are defined by the user)
min_x = -1
max_x = 1

min_y = -1
max_y = 1

z_value = 1
z_value_transformation = 2

# Create a grid of x and y values that will be the points of our "plane"
x_plane = np.linspace(min_x, max_x, num_points)
y_plane = np.linspace(min_y, max_y, num_points)

X_grid, Y_grid = np.meshgrid(x_plane, y_plane)
X_grid = X_grid.reshape((-1, 1))
Y_grid = Y_grid.reshape((-1, 1))
Z_grid = np.full(X_grid.shape, z_value)

# Obtain the generated points in a 3D space
points_3D_original = np.concatenate((X_grid, Y_grid, Z_grid), axis=1)

# Obtain the 2D points by removing the third dimension
points_2D_original = get_rid_of_z_dimension(points_3D_original)

# Create our custom transformation matrices
affine_transformation = create_affine_transformation()
homography = create_homography_transformation()
print(f'Our homography matrix:')
print(homography)

# Apply our affine transformation to the 3D object
points_3D_affine = apply_transformation(points_3D_original, affine_transformation)
print("points_3D_original.shape", points_3D_original.shape)

# Apply our homography transformation to the 3D object
points_3D_homography = apply_transformation(points_3D_original, homography)
homography_determinant = np.linalg.det(homography)
print(f"Our homography's determinant: {homography_determinant}")

# Remove the third dimension of our 3D affine transformation
points_2D_affine = get_rid_of_z_dimension(points_3D_affine)

# Remove the third dimension of our 3D homography transformation
points_2D_homography = get_rid_of_z_dimension(points_3D_homography)

Our homography matrix:
[[ 6.123234e-17 -1.000000e+00  0.000000e+00]
 [ 1.000000e+00  6.123234e-17  0.000000e+00]
 [ 1.000000e+00  3.000000e+00  1.000000e+00]]
transformation.shape:  (3, 3)
input.shape:  (2500, 3)
points_3D_original.shape (2500, 3)
transformation.shape:  (3, 3)
input.shape:  (2500, 3)
Our homography's determinant: 1.0


#### We cannot plot these transformations just yet since we haven't defined a way for us to retrieve the homogeneous coordinates of an object between different planes.

#### Let's create a function that can allow us to apply these homogeneous coordinates to our image(s)!

In [24]:
DEFAULT_HOMOGENOUS_COORDINATE = 1           # User-selected homogeneous coordinate (default)

def get_homogenous_img_coordinates(img):
    '''
        This function derives the homogeneous coordinates of an image or particular object of interest in order to
        apply homography transformations.

        Params: img (an image tensor containing the object of interest).
        Returns: the new image coordinates of the input image as a tensor after adding in the homogeneous coordinates.
    '''

    (width, height) = img.shape
    X, Y = np.mgrid[0:width, 0:height]

    X = X.reshape((-1, 1))
    Y = Y.reshape((-1, 1))

    homogenous_coordinates = np.full(X.shape, DEFAULT_HOMOGENOUS_COORDINATE)

    # Add the homogeneous coordinates to the end of the image matrix/vector
    img_coordinates = np.concatenate((X, Y, homogenous_coordinates), axis=1)
    
    print("img_coordinates.shape: ",img_coordinates.shape)
    return img_coordinates

#### Now that we have our newly generated image coordinates, we can create functions that will plot our new image in a 3D or 2D space.

In [ ]:
SCATTER_DOT_SIZE = 5

def plot_image_3D(img_coordinates, img_color):
    '''
        This function plots an image into a 3D plane.
        This function depends on deriving the new image coordinates (after applying the homography coordinates to the image).

        Params: img_coordinates (the image coordinates including the homography coordinates at its end).
                img_color (the color of the image that we desire).
        Returns: None (but displays the image onto a 3D plane).
    '''

    cmap_colors = [(1, 'red'), (1, 'blue'), (1, 'green')]  # Colors for the combined dimension
    custom_cmap = LinearSegmentedColormap.from_list('custom', cmap_colors)

    # Prepare the plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Fill in the image onto the 3D plane point-by-point
    ax.scatter(img_coordinates[:,0],img_coordinates[:,1], img_coordinates[:,2], c=img_color, cmap=plt.cm.gray, s=SCATTER_DOT_SIZE) # lineWidth parameter causes errors (DON'T USE!)
    plt.show()


def plot_image_2D(img_coordinates, img_color):
    '''
        This function plots an image into a 2D plane.
        This function depends on deriving the new image coordinates (after applying the homography coordinates to the image).

        Params: img_coordinates (the image coordinates including the homography coordinates at its end).
                img_color (the color of the image that we desire).
        Returns: None (but displays the image onto a 2D plane).
    '''

    cmap_colors = [(1, 'red'), (1, 'blue'), (1, 'green')]  # Colors for the combined dimension
    custom_cmap = LinearSegmentedColormap.from_list('custom', cmap_colors)

    # Prepare the plot
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    # Fill in the image onto the 3D plane point-by-point
    ax.scatter(img_coordinates[:,0],img_coordinates[:,1], c=img_color, cmap=plt.cm.gray, s=SCATTER_DOT_SIZE) # lineWidth parameter causes errors (DON'T USE!)
    plt.show()

#### We have shown how we can plot images onto a new plane after incorporating homography coordinates to the image itself.

#### However, if we want to analyze the projection process on a deeper level, we can't really see that by simply just plotting image tensors. To tackle this problem, we can create functions that help us project these adjusted image coordinates onto a plane, specifically in a 2D space!

#### Projecting a tensor onto a plane requires normalization of the tensor itself, which we will define first.

In [25]:
def get_normal_vector(plane_coordinates):
    '''
        This function computes the coordinates of a normal vector in a particular plane.

        Params: plane_coordinates (3 points chosen from a plane of interest).
        Returns: a normal unit vector within the same input plane.
    '''

    # We compute the equation for the normal vector of a plane from 3 points
    # by using the cross product
    coord1 = plane_coordinates[1,:]
    coord2 = plane_coordinates[2,:]
    coord3 = plane_coordinates[3,:]
    
    vector1 = coord1-coord2 
    vector2 = coord2-coord3

    normal = np.cross(vector1, vector2)
    normal_unit_vec = normal / np.linalg.norm(normal)
    return normal_unit_vec

#### With this function, we can now retrieve a 2D projection using the homogeneous coordinates we derived earlier.

In [ ]:
def project_to_2D(homog_coord):
    '''
        This function retrieves a 2D projection of a particular image in a 2D plane.
        We can simply perform this operation by dividing the homogeneous coordinates by the z-dimension. 
        
        Note: we are assuming to project onto a plane where the rays would go through the origin
              and there's only one set of planes where we can do this (the planes centered around and facing the origin).

              We also expect the input homogeneous coordinates to be of the form (N*M)x3 where N is the number of rows
              and M is the number of columns of the image. Hence, the resulting projection should have N*M rows and 3 columns.
            
        Params: the homogeneous coordinates of an image.
        Returns: a 2D projection vector of our image.
    '''
    
    Z_DIM_INDEX = 2

    z_dimension = homog_coord[:,Z_DIM_INDEX].reshape(-1, 1)
    projection_2D = homog_coord/z_dimension     # Simply divide the homogeneous coords by the z-dimension

    return projection_2D

#### So far, we have defined functions that applies a transformation to an image and projects the image from a 3D -> 2D space.

#### What if we want to reverse the transformations applied to our image (to return to its original state)?

#### We can develop a series of metrics and functions to apply a reverse transform!

#### The first step requires retrieving the min and max sequence of cells in order to obtain the limits (or bounding box) of the transformations.

In [ ]:
def get_min_and_max(transformation_output):
    '''
        This function retrieves the maximum and minimum coordinates of the transformed image coordinates. 
        This is a helper function to the reverse transformation method defined below.

        Params: transformation_output (a numpy array - could be 2D or 3D - that represents the coordinates of our transformed image).
        Returns: two x, y coordinates - one representing the minimum x, y pair and the other representing the max x, y pair.
    '''

    # Rows > Cols
    if transformation_output.shape[0] > transformation_output.shape[1]:
        min_x = np.min(transformation_output[:,0]).astype(int)
        max_x = np.max(transformation_output[:,0]).astype(int)

        min_y = np.min(transformation_output[:,1]).astype(int)
        max_y = np.max(transformation_output[:,1]).astype(int)

    # Cols >= Rows
    else:
        min_x = np.min(transformation_output[0,:]).astype(int)
        max_x = np.max(transformation_output[0,:]).astype(int)

        min_y = np.min(transformation_output[1,:]).astype(int)
        max_y = np.max(transformation_output[1,:]).astype(int)
    
    # Return coords in tuple fashion
    min_tuple = (min_x, min_y)
    max_tuple = (max_x, max_y)

    return min_tuple, max_tuple

#### We have the max and min coordinates of our transformed image. Let us define our reverse transformation method using `Raster Scan Reverse Transform`.

In [ ]:
def raster_scan_reverse_transform(original_image, homogenous_input, homogenous_color_map, transform):
    '''
        This function applies the Raster Scan Reverse Transform to revert the transformation effects of our image.

    '''

    """
    Takes a set up of homogenous coordinates, a color map for those coordinates,
    and a transform to apply to those coordinates. Applies the transform to get 
    the output coordinates and gets a pair of the min x & y values and the max 
    x & y values (after dividing by the homogenous coordinate). It then 
    raster scans through all values in between (like its scanning through
    a bounding box) and applies the reverse transform. We then get the color
    value at the reverse transform location and give the coordinate in our output
    space that color value.
    Parameters:
        - original_image (np.array): original input, of the form h*w (height*width)
        - homogenous_input (np.array): coordinates of original input, of the form 
            height*width*3 (3 is for x, y, and z coord)
        - homogenous_colormap (np.array): maps a color to each of the indices in
            homogenous_input. of the form height*width*1
        - transform (np.array): the transform to apply to the homogenous_input
    Returns:
        - homogenous_output (np.array) the output coordinates that we raster scanned through
        - homogenous_output_color_map (np.array) the color map for the output
    """

    # Apply the transform to the homogenous_input
    output = apply_transformation(homogenous_input, transform)
    # Divide by the z coordinate
    output_homography = project_to_2D(output)
    print("Output shape: ", output.shape)
    print("output_homography shape: ", output_homography.shape)
    # Bottom left and top right of bounding box to rasterize
    bottom_left, top_right = get_min_and_max(output_homography)
    print("bottom_left", bottom_left)
    print("top_right", top_right)
    # Iterate through from bottom_left to top_right